# Nanodegree Engenheiro de Machine Learning
## Aprendizagem Supervisionada
## Project 2: Construindo um Sistema de Intervenção para Estudantes

Bem-vindo ao segundo projeto do Nanodegree de Machine Learning! Neste Notebook, alguns templates de código já foram fornecidos, e será o seu trabalho implementar funcionalidades necessárias para completar este projeto com êxito. Seções que começam com **'Implementação'** no cabeçalho indicam que o bloco de código que se segue precisará de funcionalidades adicionais que você deve fornecer. Instruções serão providenciadas para cada seção e as especificações para cada implementação estarão marcadas no bloco de código com o comando `'TODO'`. Tenha certeza de ler atentamente todas as instruções!

Além do código implementado, haverá questões relacionadas ao projeto e à implementação que você deve responder. Cada seção em que você tem que responder uma questão será antecedida de um cabeçalho **'Questão X'**. Leia atentamente cada questão e escreva respostas completas nas caixas de texto subsequentes que começam com **'Resposta: '**. O projeto enviado será avaliado baseado nas respostas para cada questão e a implementação que você forneceu.  

>**Nota:** Células de código e Markdown podem ser executadas utilizando o atalho de teclado **Shift + Enter**. Além disso, as células Markdown podem ser editadas, um clique duplo na célula entra no modo de edição.

### Questão 1 - Classificação versus Regressão
*Seu objetivo neste projeto é identificar estudantes que possam precisar de intervenção antecipada antes de serem reprovados. Que tipo de problema de aprendizagem supervisionada é esse: classificação ou regressão? Por quê?*

**Resposta: ** É um problema de classificação,pois devemos fornecer uma respota binária ou seja separar os dados em classes, verdadeiro e falso.

## Observando os Dados
Execute a célula de código abaixo para carregar as bibliotecas de Python necessárias e os dados sobre os estudantes. Note que a última coluna desse conjunto de dados, `'passed'`, será nosso rótulo alvo (se o aluno foi ou não aprovado). As outras colunas são atributos sobre cada aluno.

In [4]:
# Importar bibliotecas
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Ler os dados dos estudantes
student_data = pd.read_csv("student-data.csv")
print "Os dados dos estudantes foram lidos com êxito!"

Os dados dos estudantes foram lidos com êxito!


### Implementação: Observando os Dados
Vamos começar observando o conjunto de dados para determinar quantos são os estudantes sobre os quais temos informações e entender a taxa de graduação entre esses estudantes. Na célula de código abaixo, você vai precisar calcular o seguinte:
- O número total de estudantes, `n_students`.
- O número total de atributos para cada estudante, `n_features`.
- O número de estudantes aprovados, `n_passed`.
- O número de estudantes reprovados, `n_failed`.
- A taxa de graduação da classe, `grad_rate`, em porcentagem (%).


In [6]:
data_f_shape=student_data.shape


# TODO: Calcule o número de estudante
n_students = data_f_shape[0]

# TODO: Calcule o número de atributos
n_features = data_f_shape[1]-1


# TODO: Calcule o número de alunos aprovados
n_passed = student_data[student_data["passed"]=="yes"].shape[0]

# TODO: Calcule o número de alunos reprovados
n_failed = student_data[student_data["passed"]=="no"].shape[0]

# TODO: Calcule a taxa de graduação
grad_rate = (float(n_passed)/float(n_students))*100

# Imprima os resultados
print "Número total de estudantes: {}".format(n_students)
print "Número de atributos: {}".format(n_features)
print "Número de estudantes aprovados: {}".format(n_passed)
print "Número de estudantes reprovados: {}".format(n_failed)
print "Taxa de graduação: {:.2f}%".format(grad_rate)

Número total de estudantes: 395
Número de atributos: 31
Número de estudantes aprovados: 265
Número de estudantes reprovados: 130
Taxa de graduação: 67.09%


## Preparando os Dados
Nesta seção, vamos preparara os dados para modelagem, treinamento e teste.

### Identificar atributos e variáveis-alvo
É comum que os dados que você obteve contenham atributos não numéricos. Isso pode ser um problema, dado que a maioria dos algoritmos de machine learning esperam dados númericos para operar cálculos.

Execute a célula de código abaixo para separar os dados dos estudantes em atributos e variáveis-alvo e verificar se algum desses atributos é não numérico.

In [64]:
# Extraia as colunas dos atributo
feature_cols = list(student_data.columns[:-1])

# Extraia a coluna-alvo, 'passed'
target_col = student_data.columns[-1] 

# Mostre a lista de colunas
print "Colunas de atributos:\n{}".format(feature_cols)
print "\nColuna-alvo: {}".format(target_col)

# Separe os dados em atributos e variáveis-alvo (X_all e y_all, respectivamente)
X_all = student_data[feature_cols]
y_all = student_data[target_col].map({'yes': 1, 'no': 0})

# Mostre os atributos imprimindo as cinco primeiras linhas
print "\nFeature values:"
print X_all.head()

Colunas de atributos:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Coluna-alvo: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Pré-processar Colunas de Atributo

Como você pode ver, há muitas colunas não numéricas que precisam ser convertidas! Muitas delas são simplesmente `yes`/`no`, por exemplo, a coluna `internet`. É razoável converter essas variáveis em valores (binários) `1`/`0`.

Outras colunas, como `Mjob` e `Fjob`, têm mais do que dois valores e são conhecidas como variáveis categóricas. A maneira recomendada de lidar com esse tipo de coluna é criar uma quantidade de colunas proporcional aos possíveis valores (por exemplo, `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc), e assinalar `1` para um deles e `0` para todos os outros.

Essas colunas geradas são por vezes chamadas de _variáveis postiças_ (em inglês: _dummy variables_), e nós iremos utilizar a função [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) para fazer essa conversão. Execute a célula de código abaixo para executar a rotina de pré-processamento discutida nesta seção.

In [65]:
def preprocess_features(X):
    ''' Pré-processa os dados dos estudantes e converte as variáveis binárias não numéricas em
        variáveis binárias (0/1). Converte variáveis categóricas em variáveis postiças. '''
    
    # Inicialize nova saída DataFrame
    output = pd.DataFrame(index = X.index)

    # Observe os dados em cada coluna de atributos 
    for col, col_data in X.iteritems():
        
        # Se o tipo de dado for não numérico, substitua todos os valores yes/no por 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # Se o tipo de dado for categórico, converta-o para uma variável dummy
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Reúna as colunas revisadas
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementação: Divisão dos Dados de Treinamento e Teste
Até agora, nós convertemos todos os atributos _categóricos_ em valores numéricos. Para o próximo passo, vamos dividir os dados (tanto atributos como os rótulos correspondentes) em conjuntos de treinamento e teste. Na célula de código abaixo, você irá precisar implementar o seguinte:
- Embaralhe aleatoriamente os dados (`X_all`, `y_all`) em subconjuntos de treinamento e teste.
  - Utilizar 300 pontos de treinamento (aproxidamente 75%) e 95 pontos de teste (aproximadamente 25%).
  - Estabelecer um `random_state` para as funções que você utiliza, se a opção existir.
  - Armazene os resultados em `X_train`, `X_test`, `y_train` e `y_test`.

In [66]:
# TODO: Importe qualquer funcionalidade adicional de que você possa precisar aqui
from sklearn.model_selection import train_test_split

# TODO: Estabeleça o número de pontos de treinamento
num_train = 95

# Estabeleça o número de pontos de teste
num_test = X_all.shape[0] - num_train

X_train ,X_test ,y_train , y_test = train_test_split(X_all, y_all, test_size=0.24, random_state=42)

# TODO: Emabaralhe e distribua o conjunto de dados de acordo com o número de pontos de treinamento e teste abaixo

# Mostre o resultado da distribuição
print "O conjunto de treinamento tem {} amostras.".format(X_train.shape[0])
print "O conjunto de teste tem {} amostras.".format(X_test.shape[0])

O conjunto de treinamento tem 300 amostras.
O conjunto de teste tem 95 amostras.


## Treinando e Avaliando Modelos
Nesta seção, você irá escolher 3 modelos de aprendizagem supervisionada que sejam apropriados para esse problema e que estejam disponíveis no `scikit-learn`. Primeiro você irá discutir o raciocínio por trás da escolha desses três modelos considerando suas vantagens e desvantagens e o que você sabe sobre os dados. Depois você irá ajustar o modelo a diferentes tamanhos de conjuntos de treinamento (com 100, 200 e 300 pontos) e medir a pontuação F<sub>1</sub>. Você vai precisar preencher três tabelas (uma para cada modelo) que mostrem o tamanho do conjunto de treinamento, o tempo de treinamento, o tempo de previsão e a pontuação F<sub>1</sub> no conjunto de treinamento.

**Os seguintes modelos de aprendizagem supervisionada estão atualmente disponíveis no **[`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html)** para você escolher:**
- Gaussian Naive Bayes (GaussianNB)
- Árvores de Decisão
- Métodos de agregação (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Método do gradiente estocástico (SGDC)
- Máquinas de vetores de suporte (SVM)
- Regressão logística

### Questão 2 - Aplicação dos Modelos
*Liste três modelos de aprendizagem supervisionada que são apropriadas para esse problema. Para cada modelo escolhido:*
- Descreva uma aplicação em mundo real na indústria em que o modelo pode ser aplicado. *(Talvez você precise fazer um pouco de pesquisa para responder essa questão – dê as devidas referências!)* 
- Quais são as vantagens do modelo; quando ele tem desempenho melhor? 
- Quais são as desvantagens do modelo, quando ele tem desempenho pior?
- O que faz desse modelo um bom candidato para o problema, considerando o que você sabe sobre os dados?


** Resposta **

## Naive Bayes (GaussianNB)
## Descreva uma aplicação em mundo real na indústria em que o modelo pode ser aplicado.
Filtragem de spam, caso clássico de uso do algoritmo.

## Quais são as vantagens do modelo; quando ele tem desempenho melhor?
Precisa de poucos dados para treinamento. Fácil implementação. Funciona em grande parte dos problemas de classficação.
Rápida execução.

## Quais são as desvantagens do modelo, quando ele tem desempenho pior?
Não consegue mapear a depêndencia entre as variaveis.


## O que faz desse modelo um bom candidato para o problema, considerando o que você sabe sobre os dados?
Ele é um bom classificador.
Lida bem com separação não linear. (temos 48 dimensões seria dificil garantir que os dados sejam separáveis linearmente)
Lida bem com poucos exemplos de treinamento.

Fontes:
http://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes

https://pt.slideshare.net/ashrafmath/naive-bayes-15644818

## SVM
## Descreva uma aplicação em mundo real na indústria em que o modelo pode ser aplicado. 
Classificação de imagens. Bio Informática, Classificação de proteínas e de cânceres. Reconhecimento de escrita manual

## Quais são as vantagens do modelo; quando ele tem desempenho melhor?
Funciona bem com pequenos conjuntos de treinamento. Generaliza bem o modelo. Lida com a não linearidade.
Funciona bem em espaços com muitas dimensões. Encontra um mínimo global e não um mínimo local.

## Quais são as desvantagens do modelo, quando ele tem desempenho pior?
Alta sensibilidaed a ruídos. Definição da função Kernel. Alto custo computacional.

## O que faz desse modelo um bom candidato para o problema, considerando o que você sabe sobre os dados?
Devido ao conjunto de treinamento não ser tão extensos e o número de características ser grande, esse é um bom
candidato para utilizar nesses dados.

Fontes
http://www.svms.org/

http://scikit-learn.org/stable/modules/svm.html


## AdaBoost
## Descreva uma aplicação em mundo real na indústria em que o modelo pode ser aplicado. 
	Biologia, Visão computacional,Processamento de fala.
    
## Quais são as vantagens do modelo; quando ele tem desempenho melhor?
     Baixa tendência a overfitting,	Atinge um erro mínimo global,	Tem uma boa flexibilidades, pois pode ser aplicado a vários tipos de classificadores.
	Pode ser um aprendiz rápido dependendo do algoritmo.
	
## Quais são as desvantagens do modelo, quando ele tem desempenho pior?
	Tem alta sensilidade a ruído distribuido uniformemente, não lida bem com outliers.
	
## O que faz desse modelo um bom candidato para o problema, considerando o que você sabe sobre os dados?
	Por ser um meta-algoritmo que utiliza uma série de outras instâncias de classificadores ou weak learners, apresenta bastante
		flexibilidade e irá me permitir procurar uma configuração ideal afim de obter uma boa pontuação na classificação.
        
Fontes: 
http://www.nickgillian.com/wiki/pmwiki.php/GRT/AdaBoost

http://user.ceng.metu.edu.tr/~tcan/ceng734_f1112/Schedule/adaboost.pdf

### Configuração
Execute a célula de código abaixo para inicializar três funções de ajuda que você pode utilizar para treinar e testar os três modelos de aprendizagem supervisionada que você escolheu acima. As funções são as seguintes:
- `train_classifier` - recebe como parâmetro um classificador e dados de treinamento e ajusta o classificador aos dados.
- `predict_labels` - recebe como parâmetro um classificador ajustado, atributos e rótulo alvo e faz estimativas utilizando a pontuação do F<sub>1</sub>.
- `train_predict` - recebe como entrada um classificador, e dados de treinamento e teste, e executa `train_clasifier` e `predict_labels`.
 - Essa função vai dar a pontuação F<sub>1</sub> tanto para os dados de treinamento como para os de teste, separadamente.

In [69]:
def train_classifier(clf, X_train, y_train):
    ''' Ajusta um classificador para os dados de treinamento. '''
    
    # Inicia o relógio, treina o classificador e, então, para o relógio
    start = time()    
    clf.fit(X_train, y_train)
    end = time()
    
    # Imprime os resultados
    print "O modelo foi treinado em {:.4f} segundos".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Faz uma estimativa utilizando um classificador ajustado baseado na pontuação F1. '''
    
    # Inicia o relógio, faz estimativas e, então, o relógio para
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Imprime os resultados de retorno
    print "As previsões foram feitas em {:.4f} segundos.".format(end - start)
    return f1_score(target.values, y_pred, pos_label=1)


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Treina e faz estimativas utilizando um classificador baseado na pontuação do F1. '''
    
    # Indica o tamanho do classificador e do conjunto de treinamento
    print "Treinando um {} com {} pontos de treinamento. . .".format(clf.__class__.__name__, len(X_train))
    
    # Treina o classificador
    train_classifier(clf, X_train, y_train)
    
    # Imprime os resultados das estimativas de ambos treinamento e teste
    print "Pontuação F1 para o conjunto de treino: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "Pontuação F1 para o conjunto de teste: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementação: Métricas de Desempenho do Modelo
Com as funções acima, você vai importar os três modelos de aprendizagem supervisionada de sua escolha e executar a função `train_prediction` para cada um deles. Lembre-se de que você vai precisar treinar e usar cada classificador para três diferentes tamanhos de conjuntos de treinamentos: 100, 200 e 300 pontos. Então você deve ter 9 saídas diferentes abaixo – 3 para cada modelo utilizando cada tamanho de conjunto de treinamento. Na célula de código a seguir, você deve implementar o seguinte:
- Importe os três modelos de aprendizagem supervisionada que você escolheu na seção anterior.
- Inicialize os três modelos e armazene eles em `clf_A`, `clf_B` e `clf_C`.
 - Defina um `random_state` para cada modelo, se a opção existir.
 - **Nota:** Utilize as configurações padrão para cada modelo – você vai calibrar um modelo específico em uma seção posterior.
- Crie diferentes tamanhos de conjuntos de treinamento para treinar cada modelo.
 - *Não embaralhe e distribua novamente os dados! Os novos pontos de treinamento devem ser tirados de `X_train` e `y_train`.*
- Treine cada modelo com cada tamanho de conjunto de treinamento e faça estimativas com o conjunto de teste (9 vezes no total).  
**Nota:** Três tabelas são fornecidas depois da célula de código a seguir, nas quais você deve anotar seus resultados.

In [70]:
# TODO: Importe os três modelos de aprendizagem supervisionada do sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
# from sklearn import model_B
# from skearln import model_C

# TODO: Inicialize os três modelos
clf_A = GaussianNB()
clf_B = svm.SVC(random_state=17)
clf_C = AdaBoostClassifier(random_state=7)

# TODO: Configure os tamanho dos conjuntos de treinamento
X_train_100 = X_train.head(100)
y_train_100 = y_train.head(100)


X_train_200 = X_train.head(200)
y_train_200 = y_train.head(200)

X_train_300 = X_train
y_train_300 = y_train


# TODO: Executar a função 'train_predict' para cada classificador e cada tamanho de conjunto de treinamento
print('NB: 100' )
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
print('NB: 200' )
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
print('NB: 300' )
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

print('SVC: 100' )
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
print('SVC: 200' )
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
print('SVC: 300' )
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

print('boost: 100' )
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
print('boost: 200' )
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
print('boost: 300' )
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

NB: 100
Treinando um GaussianNB com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0020 segundos
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de treino: 0.8467.
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de teste: 0.8029.
NB: 200
Treinando um GaussianNB com 200 pontos de treinamento. . .
O modelo foi treinado em 0.0030 segundos
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de treino: 0.8406.
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de teste: 0.7244.
NB: 300
Treinando um GaussianNB com 300 pontos de treinamento. . .
O modelo foi treinado em 0.0010 segundos
As previsões foram feitas em 0.0010 segundos.
Pontuação F1 para o conjunto de treino: 0.8038.
As previsões foram feitas em 0.0000 segundos.
Pontuação F1 para o conjunto de teste: 0.7634.
SVC: 100
Treinando um SVC com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0020 segundos
As previsões 

### Resultados Tabulados
Edite a célula abaixo e veja como a tabela pode ser desenhada em [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). Você deve salvar seus resultados abaixo nas tabelas fornecidas.

** Classificador 1 - GaussianNB  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |        0.0070             |      0.0010                       |    0.8467                        |             0.8029         |
| 200                                |        0.0030        |               0.0010              |         0.8406                   |              0.7244        |
| 300                                |        0.0020              |         0.0010                    |    0.8038                        |        0.7634       |

** Classificador 2 - SVM  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |         0.0040             |         0.0010                    |   0.8777                         |    0.7746                  |
| 200                                |       0.0040        |                0.0010             |          0.8679                  |             0.7815         |
| 300                                |           0.0090           |             0.0020                |     0.8761                       |        0.7838       |

** Classificador 3 - Ada Boost  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |         0.1640              |     0.0030                        |   0.9481                         |    0.7669                  |
| 200                                |        0.1120              |       0.0040                       |    0.8927                        |     0.8281                 |
| 300                                |        0.1300              |       0.0030                       |    0.8637                        |      0.7820                |

## Escolhendo o Melhor Modelo
Nesta seção final, você irá escolher dos três modelos de aprendizagem supervisionada o *melhor* para utilizar os dados dos estudantes. Você então executará um busca em matriz otimizada para o modelo em todo o conjunto de treinamento (`X_train` e `y_train`) ao calibrar pelo menos um parâmetro, melhorando em comparação a pontuação F<sub>1</sub> do modelo não calibrado. 

### Questão 3 - Escolhendo o Melhor Modelo
*Baseando-se nos experimentos que você executou até agora, explique em um ou dois parágrafos ao conselho de supervisores qual modelo que você escolheu como o melhor. Qual modelo é o mais apropriado baseado nos dados disponíveis, recursos limitados, custo e desempenho?*

**Resposta: **
Ao avaliar a execução dos modelos posso afirmar que em questão de precisão no conjunto de testes não houve muita variância, sendo que do menor valor GaussianNB c\ 200 exemplos: 0,7244 para o maior Ada Bost c\ 200 exemplos: 0,8281 é uma variância de 0,1037 que na minha opinião é um valor médio, no entanto se observarmos uma média da pontuação de cada modelo para as 3 quantidades de exemplos de treinamento chegaremos aos seguintes valores: Gaussian NB: 0,7635, SVM: 0,7799, Ada Boost: 0,7923 que são na minha opinião são bem próximos, ao analisar os tempos, na parte de estimativa(teste) todos tiveram tempo na casa de centéssimos de miléssimos de segundos, isso para 100 exemplos, o que me leva a crer que nesse quesito os 3 algoritmos estão em par de igualdade no sentido da utilização prática, na questão de treinamento todos tiveram tempos extramamente baixos, mas se olharmos proporcionamente o método adaboost foi até 41 vezes mais lento no treinamento, o que é não é um problema para esse conjunto de treinamento, mas pode ser em casos com mais exemplos. 
Agora considerando a próxima tarefa a ser feita, calibrar o modelo com 300 exemplos, o que obteve melhor desempenho nessa situação no conjunto de testes foi o SVM, entretando a diferença para o segundo (Adaboost) foi de apenas 0,018% mas na média ele foi 0,124% superior. 
Levando tudo em consideração minha intuição, me diz para escolher o AdaBoost, pois seu ponto fraco (Tempo de treinamento) não é impactante no conjunto de dados atuais, sua média de acerto nos teste é a melhor de todas e seu acerto nos teste ao ser treinado com todos 300 os exemplos é insignificantemente menor ao primeiro colocado nessa situação.

### Questão 4 – O Modelo para um Leigo
*Em um ou dois parágrafos, explique para o conselho de supervisores, utilizando termos leigos, como o modelo final escolhido deve trabalhar. Tenha certeza que você esteja descrevendo as melhores qualidades do modelo, por exemplo, como o modelo é treinado e como ele faz uma estimativa. Evite jargões técnicos ou matemáticos, como descrever equações ou discutir a implementação do algoritmo.*

**Resposta: **
Para explicar nosso modelo usarei uma analogia que acho válida com o mundo jurídico:
Nosso modelo faz estimativas como se houvesse um concelho de juízes e cada um desses juízes somente proferisse a sentença correta em pouco mais da metade dos casos, mas no ententanto contamos a escolha de cada um como um voto e então ficamos com a opinião da maioria, nós provavelmente teremos uma setença muito mais confiável. Preliminarmente para que os juízes possam formar suas opiniões, nós fornecemos a eles informações sobre casos anteriores e pedimos sua opinião para comparar com as sentenças corretas, e sempre que ele incorrem em erro em determinado caso, esse caso então passa a ser mais provavel de ser escolhido para que o juíz analise novamente, assim podendo perceber nuances que não havia notado na sua primeira análise.

### Implementação: Calibrando o Modelo
Calibre o modelo escolhido. Utilize busca em matriz (`GridSearchCV`) com, pelo menos, um parâmetro importante calibrado com, pelo menos, 3 valores diferentes. Você vai precisar utilizar todo o conjunto de treinamento para isso. Na célula de código abaixo, você deve implementar o seguinte:
- Importe [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) e [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Crie o dicionário de parâmetros que você deseja calibrar para o modelo escolhido.
 - Examplo: `parameters = {'parameter' : [list of values]}`.
- Inicialize o classificador que você escolheu e armazene-o em `clf`.
- Crie a função de pontuação F<sub>1</sub> utilizando `make_scorer` e armazene-o em `f1_scorer`.
 - Estabeleça o parâmetro `pos_label` para o valor correto!
- Execute uma busca em matriz no classificador `clf` utilizando o `f1_scorer` como método de pontuação e armazene-o em `grid_obj`.
- Treine o objeto de busca em matriz com os dados de treinamento (`X_train`, `y_train`) e armazene-o em `grid_obj`.

In [109]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
# TODO: Importe 'GridSearchCV' e 'make_scorer'

# TODO: Crie a lista de parâmetros que você gostaria de calibrar
parameters  = {'learning_rate':[0.1,0.2,0.3,0.5, 1,2], 
               'n_estimators':[5,7,8,9,10,11,12,13,14,15,20,100]}

# TODO: Inicialize o classificador
clf = AdaBoostClassifier(random_state=7)

# TODO: Faça uma função de pontuação f1 utilizando 'make_scorer' 
f1_scorer = make_scorer(f1_score, greater_is_better=True)

# TODO: Execute uma busca em matriz no classificador utilizando o f1_scorer como método de pontuação
grid_obj = GridSearchCV(estimator=clf,param_grid=parameters,scoring=f1_scorer)


# TODO: Ajuste o objeto de busca em matriz para o treinamento de dados e encontre os parâmetros ótimos
grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_
# Reporte a pontuação final F1 para treinamento e teste depois de calibrar os parâmetrosprint "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de treinamento.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de teste.".format(predict_labels(clf, X_test, y_test))

13
0.1
As previsões foram feitas em 0.0020 segundos.
O modelo calibrado tem F1 de 0.8299 no conjunto de treinamento.
As previsões foram feitas em 0.0010 segundos.
O modelo calibrado tem F1 de 0.8000 no conjunto de teste.


### Questão 5 - Pontuação F<sub>1</sub> Final
*Qual é a pontuação F<sub>1</sub> do modelo final para treinamento e teste? Como ele se compara ao modelo que não foi calibrado?*

**Resposta: **
A pontuação foi de 0,8 ou seja ainda mais próximo de ser perfeito,se relaciona como um modelo um pouco menos complexo, com menos weak learners, mas que foi mais capaz de realizar acertos no conjunto de testes.

> **Nota**: Uma vez que você completou todas as implementações de código e respondeu todas as questões acima com êxito, você pode finalizar seu trabalho exportando o iPython Nothebook como um document HTML. Você pode fazer isso utilizando o menu acima e navegando para  
**File -> Download as -> HTML (.html)**. Inclua a documentação final junto com o notebook para o envio do seu projeto.